# Clustering Crypto

In [50]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
from pathlib import Path
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [51]:
# Alternatively, use the provided csv file:
file_path = Path("crypto_data.csv")
# Create a DataFrame
df_crypto = pd.read_csv(file_path, encoding="ISO-8859-1")
df_crypto.head(10)

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
5,1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
6,2015,2015 coin,X11,True,PoW/PoS,NaN,0
7,BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
8,ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
9,LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


### Data Preprocessing

In [52]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
df_crypto = df_crypto.set_index('Unnamed: 0')
df_crypto

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,,
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000


In [53]:
# Keep only cryptocurrencies that are trading
df_crypto = df_crypto.loc[df_crypto['IsTrading']==True]
df_crypto

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,,
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610


In [54]:
# Keep only cryptocurrencies with a working algorithm
df_crypto['Algorithm'].unique()
# No crypto with non working algorithm as there is no null or N/A value

array(['Scrypt', 'X11', 'SHA-256', 'X13', 'Ethash', 'CryptoNight-V7',
       'Equihash', 'SHA-512', 'Multiple', 'X15', 'NIST5', 'Quark',
       'Groestl', 'PoS', 'NeoScrypt', 'SHA3', 'HybridScryptHash256',
       'Scrypt-n', 'PHI1612', 'Lyra2REv2', 'CryptoNight', 'Shabal256',
       'Counterparty', 'Blake', 'Momentum', 'Stanford Folding', 'QuBit',
       'XG Hash', 'M7 POW', 'Curve25519', 'Lyra2RE', 'QUAIT', 'vDPOS',
       'Blake2b', 'BLAKE256', '1GB AES Pattern Search', 'Dagger',
       'CryptoNight-Lite', 'X11GOST', 'SHA-256D', 'POS 3.0',
       'Progressive-n', 'DPoS', 'Lyra2Z', 'X14', 'Time Travel', 'Argon2',
       'Keccak', 'Blake2S', 'Dagger-Hashimoto', '536', 'Argon2d',
       'Cloverhash', 'Skein', 'SkunkHash v2 Raptor',
       'VeChainThor Authority', 'Ouroboros', 'POS 2.0', 'SkunkHash',
       'C11', 'Proof-of-BibleHash', 'SHA-256 + Hive',
       'Proof-of-Authority', 'XEVAN', 'VBFT', 'YescryptR16', 'IMesh',
       'Green Protocol', 'Semux BFT consensus', 'X16R', 'Tribus',


In [55]:
# Remove the "IsTrading" column
df_crypto = df_crypto.drop(columns=['IsTrading'])
df_crypto

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...
SERO,Super Zero,Ethash,PoW,NaN,1000000000
UOS,UOS,SHA-256,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610


In [56]:
# Remove rows with at least 1 null value
df_crypto.dropna(axis=0,inplace=True)
df_crypto

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610


In [57]:
# Remove rows with cryptocurrencies having no coins mined
df_crypto = df_crypto[df_crypto['TotalCoinsMined']>0]
df_crypto

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610


In [58]:
# Drop rows where there are 'N/A' text values
for column in df_crypto.columns:
    df_crypto = df_crypto[df_crypto[column]!='N/A']
df_crypto

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610


In [59]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
df_coinname = df_crypto['CoinName']
df_coinname.index=df_crypto.index
df_coinname.head()

Unnamed: 0
42        42 Coin
404       404Coin
1337    EliteCoin
BTC       Bitcoin
ETH      Ethereum
Name: CoinName, dtype: object

In [60]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
df_crypto=df_crypto.drop(columns=['CoinName'])
df_crypto

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610


In [61]:
# Create dummy variables for text features
df_crypto_dummy = pd.get_dummies(df_crypto, columns=["Algorithm", "ProofType"])
df_crypto_dummy

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,2.000000e+09,2000000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GAP,1.493105e+07,250000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BDX,9.802226e+08,1400222610,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
# Standardize data
crypto_scaled = StandardScaler().fit_transform(df_crypto_dummy)
print(crypto_scaled[0:1])

[[-0.11710817 -0.1528703  -0.0433963  -0.0433963  -0.0433963  -0.06142951
  -0.07530656 -0.0433963  -0.06142951 -0.06142951 -0.0433963  -0.0433963
  -0.19245009 -0.06142951 -0.09740465 -0.0433963  -0.11547005 -0.07530656
  -0.0433963  -0.0433963  -0.15191091 -0.0433963  -0.13118084 -0.0433963
  -0.0433963  -0.08703883 -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.06142951 -0.0433963  -0.08703883 -0.08703883 -0.08703883 -0.0433963
  -0.13118084 -0.13840913 -0.13840913 -0.0433963  -0.06142951 -0.0433963
  -0.07530656 -0.18168574 -0.0433963  -0.0433963  -0.0433963  -0.07530656
  -0.15826614 -0.31491833 -0.0433963  -0.08703883 -0.07530656 -0.06142951
   1.38675049 -0.0433963  -0.0433963  -0.06142951 -0.0433963  -0.0433963
  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.39879994 -0.0433963  -0.18168574 -0.0433963  -0.08703883 -0.08703883
  -0.10680283 -0.0433963  -0.13118084 -0.0433963  -0.0433963  -0.0433963
  -0.0433963  -0.07530656 -0.43911856 -0.04339

### Reducing Dimensions Using PCA

In [63]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)

# Get two principal components for the data.
crypto_pca = pca.fit_transform(crypto_scaled)

In [64]:
# Create a DataFrame with the principal components data
df_crypto_pca = pd.DataFrame(
    data=crypto_pca, columns=["PC1", "PC2","PC3"]
)
df_crypto_pca.index=df_crypto.index
df_crypto_pca.head()

,PC1,PC2,PC3
Unnamed: 0,,,
42,-0.340618,1.072830,-0.463072
404,-0.324026,1.073217,-0.463167
1337,2.303099,1.729166,-0.512429
BTC,-0.138458,-1.336915,0.116760
ETH,-0.157855,-2.067977,0.285630


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [67]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_crypto_pca)
    inertia.append(km.inertia_)

# Create the Elbow Curve using Altair
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

alt.Chart(df_elbow).mark_circle(size=60).encode(
    x='k',
    y='inertia',
).properties(
    title = "Elbow Chart")

alt.Chart(...)

In [69]:
from sklearn.metrics import silhouette_score

sil_score_max = -1 #this is the minimum possible score

for n_clusters in range(2,10):
    model = KMeans(n_clusters = n_clusters, init='k-means++', max_iter=100, n_init=1)
    labels = model.fit_predict(df_crypto_pca)
    sil_score = silhouette_score(df_crypto_pca, labels)
    #print("The average silhouette score for %i clusters is %0.2f" %(n_clusters,sil_score))
    if sil_score > sil_score_max:
        sil_score_max = sil_score
        best_n_clusters = n_clusters
print(f'Best n clusters is {best_n_clusters} and silhouette score of {sil_score_max}')

Best n clusters is 5 and silhouette score of 0.733874062795736


Running K-Means with `k=<your best value for k here>`

In [70]:
# Initialize the K-Means model
model = KMeans(n_clusters=best_n_clusters, random_state=0)
# Fit the model
model.fit(df_crypto_pca)
# Predict clusters
predictions = model.predict(df_crypto_pca)
# Create a new DataFrame including predicted clusters and cryptocurrencies features
df_crypto_pca["class"] = model.labels_

### Visualizing Results

#### Scatter Plot with Tradable Cryptocurrencies

In [71]:
# Scale data to create the scatter plot
#concat to bring all required columns in a single DataFrame
clustered_df = pd.concat([df_crypto,df_crypto_pca,df_coinname],axis =1,join='inner')

#Converting TotalCoinsMined & TotalCoinSupply to numeric
clustered_df[["TotalCoinsMined","TotalCoinSupply"]]=clustered_df[["TotalCoinsMined",
                                                                  "TotalCoinSupply"]].apply(pd.to_numeric)
#Scaling TotalCoinsMined & TotalCoinSupply 
clustered_df["TotalCoinsMined"]=clustered_df["TotalCoinsMined"]/1000000
clustered_df["TotalCoinSupply"]=clustered_df["TotalCoinSupply"]/1000000

#Show sample data
clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,class,CoinName
Unnamed: 0,,,,,,,,,
42,Scrypt,PoW/PoS,0.000042,0.000042,-0.340618,1.072830,-0.463072,1,42 Coin
404,Scrypt,PoW/PoS,1055.184902,532.000000,-0.324026,1.073217,-0.463167,1,404Coin
1337,X13,PoW/PoS,29279.424623,314159.265359,2.303099,1.729166,-0.512429,4,EliteCoin
BTC,SHA-256,PoW,17.927175,21.000000,-0.138458,-1.336915,0.116760,0,Bitcoin
ETH,Ethash,PoW,107.684223,0.000000,-0.157855,-2.067977,0.285630,0,Ethereum


In [72]:
# altair scatter plot to visualize the clusters

alt.Chart(clustered_df).mark_circle(size=60).encode(
    x='PC1',
    y='PC2',
    color='class',
    tooltip=["CoinName", "Algorithm", "TotalCoinsMined", "TotalCoinSupply"]
).interactive()

alt.Chart(...)

In [73]:
# altair scatter plot to visualize the tradable cryptocurrencies

alt.Chart(clustered_df).mark_circle(size=60).encode(
    x='TotalCoinsMined',
    y='TotalCoinSupply',
    color='class',
    tooltip=["CoinName", "Algorithm","ProofType","class"]
).interactive()

alt.Chart(...)

#### Table of Tradable Cryptocurrencies

In [74]:
# Table with tradable cryptos
display(clustered_df)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,class,CoinName
Unnamed: 0,,,,,,,,,
42,Scrypt,PoW/PoS,0.000042,0.000042,-0.340618,1.072830,-0.463072,1,42 Coin
404,Scrypt,PoW/PoS,1055.184902,532.000000,-0.324026,1.073217,-0.463167,1,404Coin
1337,X13,PoW/PoS,29279.424623,314159.265359,2.303099,1.729166,-0.512429,4,EliteCoin
BTC,SHA-256,PoW,17.927175,21.000000,-0.138458,-1.336915,0.116760,0,Bitcoin
ETH,Ethash,PoW,107.684223,0.000000,-0.157855,-2.067977,0.285630,0,Ethereum
...,...,...,...,...,...,...,...,...,...
ZEPH,SHA-256,DPoS,1999.999995,2000.000000,2.524948,0.718840,-0.016203,4,ZEPHYR
GAP,Scrypt,PoW/PoS,14.931046,250.000000,-0.338673,1.072739,-0.463073,1,Gapcoin
BDX,CryptoNight,PoW,980.222595,1400.222610,0.323842,-2.337700,0.291012,0,Beldex


In [75]:
# Print the total number of tradable cryptocurrencies
Total_Tradable_Crypto= clustered_df["CoinName"].count()
print(f"Total number of tradable cryptocurrencies are {Total_Tradable_Crypto}")

Total number of tradable cryptocurrencies are 532
